## Лабораторная 1 - Ануфриев Валентин

In [1]:
import numpy as np
import networkx as nx
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix


In [2]:
def make_synthetic_graph_dataset(
    n_graphs=200,
    n_range=(30, 50),     # число вершин в каждом графе будет случайным из этого диапазона
    p_range=(0.06, 0.12), # для ER: вероятность ребра p
    m_range=(2, 3),       # для BA: число присоединяемых рёбер m
    seed=42
):
    """
    Возвращает:
      graphs
      y меток
    Класс 0: Эрдёш–Реньи (ER), класс 1: Барбаши–Альберт (BA).
    """
    rng = np.random.default_rng(seed)
    graphs, y = [], []
    for i in range(n_graphs):
        n = int(rng.integers(n_range[0], n_range[1]+1))
        if i % 2 == 0:
            # ER граф
            p = float(rng.uniform(*p_range))
            G = nx.erdos_renyi_graph(n, p, seed=int(rng.integers(1e9)))
            y.append(0)
        else:
            # BA граф
            m = int(rng.integers(m_range[0], m_range[1]+1))
            m = max(1, min(m, n-1))
            G = nx.barabasi_albert_graph(n, m, seed=int(rng.integers(1e9)))
            y.append(1)
        # приводим к неориентированному невзвешенному виду (на всякий случай)
        G = nx.Graph(G)
        graphs.append(G)
    return graphs, np.array(y, dtype=int)

In [3]:
def _sp_length_counts(G: nx.Graph):
    """
    счтьаем словарь {длина кратчайшего пути - число пар вершин с такой длиной}
    Берём пары один раз (u < v). Длину 0 не учитываем. Недостижимые пары игнорируем
    """
    counts = {}
    for u, dist_dict in nx.shortest_path_length(G):
        for v, d in dist_dict.items():
            if v <= u:
                continue
            if d >= 1:
                counts[d] = counts.get(d, 0) + 1
    return counts  # например {1: 54, 2: 123, 3: 20}

def _build_sp_features(graphs):
    """
    Для списка графов строим матрицу признаков X размера [n_graphs, L],
    где L — макс длина кратчайшего пути, встретившаяся хотя бы где-то.
    применяется L1-норм, чтобы убрать зависимость от размера графа
    """
    all_counts = [_sp_length_counts(G) for G in graphs]
    # если графы бывают несвязные и маленькие, страхуемся: пусть L >= 1
    L = max((max(c.keys()) if c else 1) for c in all_counts)
    X = np.zeros((len(graphs), L), dtype=float)
    for i, c in enumerate(all_counts):
        for d, cnt in c.items():
            if 1 <= d <= L:
                X[i, d-1] = cnt
        s = X[i].sum()
        if s > 0:
            X[i] /= s  # L1-нормировка
    return X, L

In [4]:
# Shortest-Path Kernel 
def shortest_path_kernel(train_graphs, test_graphs):
    """
    Возвращает попарные ядра между тренировочными графами а также ядра между тестовыми и тренировочныоми где ядро - скаляр гистограмм sp длин
    """
    all_graphs = list(train_graphs) + list(test_graphs)
    X_all, _ = _build_sp_features(all_graphs)
    n_tr = len(train_graphs)
    X_tr, X_te = X_all[:n_tr], X_all[n_tr:]
    K_train = X_tr @ X_tr.T
    K_test  = X_te @ X_tr.T
    return K_train, K_test

In [5]:
#сплит, ядра, SVC, метрики
if __name__ == "__main__":

    graphs, y = make_synthetic_graph_dataset(
        n_graphs=300,
        n_range=(30, 60),
        p_range=(0.05, 0.12),
        m_range=(2, 3),
        seed=7
    )

    # Разделение
    g_tr, g_te, y_tr, y_te = train_test_split(
        graphs, y, test_size=0.3, stratify=y, random_state=0
    )

    #  Ядровые матрицы
    K_tr, K_te = shortest_path_kernel(g_tr, g_te)

    # Обучение SVC с подбором параметров
    param_grid = {"C": [1e-3, 1e-2, 1e-1, 1, 10, 100]}
    clf = GridSearchCV(
        SVC(kernel="precomputed", probability=True),
        param_grid=param_grid,
        cv=5,
        n_jobs=-1
    )
    clf.fit(K_tr, y_tr)

    #Оценка
    y_pred = clf.predict(K_te)
    y_prob = clf.predict_proba(K_te)[:, 1]

    print("Best params:", clf.best_params_)
    print("Accuracy   :", accuracy_score(y_te, y_pred))
    print("Precision  :", precision_score(y_te, y_pred))
    print("Recall     :", recall_score(y_te, y_pred))
    print("F1         :", f1_score(y_te, y_pred))
    print("ROC-AUC    :", roc_auc_score(y_te, y_prob))
    print("Confusion matrix:\n", confusion_matrix(y_te, y_pred))

Best params: {'C': 100}
Accuracy   : 0.7888888888888889
Precision  : 0.7241379310344828
Recall     : 0.9333333333333333
F1         : 0.8155339805825242
ROC-AUC    : 0.914074074074074
Confusion matrix:
 [[29 16]
 [ 3 42]]
